In [6]:
import spacy
from spacy.training.example import Example #Müssen später vermutlich was anderes nehmen
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import os
#from got_2NerTrainingData import training_data_got #sehr viele Warnings
from training_set import training_data_got #Adriana. Weniger Warnings, aber man muss komisch durch iterieren 

# Load the base model
nlp = spacy.blank("en")

# Create and add the NER pipeline component
ner = nlp.add_pipe("ner")

# Define the labels and add them to the NER component
labels = ["PERSON"]
for label in labels:
    ner.add_label(label)

#for train_data2 in training_data_got:
    # Check entity alignment
for text, annotations in training_data_got:
    doc = nlp.make_doc(text)
    #biluo_tags = offsets_to_biluo_tags(doc, annotations["entities"]) # Mit Svens Daten ging es nicht

    # Initialize the optimizer
    optimizer = nlp.initialize()

# Perform the training
for iteration in range(10):  #10 für die kleine Anzahl. Ansonsten könnte man ja so lange iterieren, bis die Fehler eine gewisse Untergrenze erreicht haben
    random.shuffle(training_data_got)
    losses = {}
    for text, annotations in training_data_got:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, sgd=optimizer)
    #print("Losses:", losses)

# Save the trained NER model
output_dir = os.getcwd()
nlp.to_disk(output_dir)

# Load the saved model
loaded_nlp = spacy.load(output_dir)


c:\Users\Adriana\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Trueborn with Cersei:{Son}
Legal, fathered by Jai..." with entities "[(522, 532, 'PERSON'), (738, 751, 'PERSON')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Adriana\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text ""
Histories & Lore:"Greyjoy Rebellion: Robb Stark..." with entities "[(40, 50, 'PERSON'), (106, 119, 'PERSON'), (215, 2...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Adriana\AppData\Local\Programs\Python\Python311\Lib\

In [9]:
with open("text.txt", "r") as f:
    text = f.read()
# Load the saved model
loaded_nlp = spacy.load(output_dir)

# Test the loaded NER model
doc = loaded_nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_) #Laut Recherche leer, weil Datensatz zu klein

Eddard Stark PERSON
Catelyn Tully PERSON
Jon Snow PERSON
Cersei Lannister PERSON
Joffrey Baratheon PERSON
Balon Greyjoy PERSON
Jon Snow PERSON
Daenerys Targaryen PERSON
Eddard Stark PERSON
Rickard Stark PERSON
Aerys II Targaryen PERSON
Robert Baratheon PERSON
Rhaegar Targaryen PERSON
Robert instead.[citationÂ PERSON
Catelyn Tully PERSON
Jon Snow PERSON
Rhaegar Targaryen PERSON
Robert Baratheon PERSON
Stannis Baratheon PERSON
Eddard Stark PERSON
Jon Snow PERSON
a Stark PERSON
Daenerys Targaryen PERSON
Drogo PERSON
Jon Snow PERSON
Renly Baratheon PERSON
Tywin Lannister PERSON
Tyrion Lannister PERSON
Gregor Clegane PERSON
Tywin Lannister PERSON
Commander Barristan PERSON
Tyrion Lannister PERSON
Daenerys Targaryen PERSON
Drogo PERSON
Jaime Lannister PERSON
Gregor Clegane PERSON
Tywin Lannister PERSON
Beric Dondarrion PERSON
Jaime. PERSON
Eddard Stark PERSON
Robb Stark PERSON
Robb Stark PERSON
Cersei Lannister PERSON
Renly Baratheon PERSON
Margaery Tyrell PERSON
Renly Baratheon PERSON
Silen

In [11]:
import json

# Laden der validierten Daten aus dem JSON-Dateiformat
with open('relationships.json', 'r') as file:
    validation_data = json.load(file)

# Ausgabe des "data" Objekts
print(validation_data)

# Iteriere über die Sätze im Validierungsdatensatz

for data in validation_data:
    sentence = data['sentence']
    entities = data['relationships']
    predicted_relationships=[]
    for entity in entities:
        firstEntity = entity['firstEntity']
        #relationshipType = entity['relationshipType'] # bei ner ja eignetlich unnötig
        secondEntity = entity['secondEntity']
        predicted_relationships.append(firstEntity)
        predicted_relationships.append(secondEntity)
    predicted_relationships = set(predicted_relationships)
    print(predicted_relationships)
    # Verarbeite den Satz mit dem geladenen NER-Modell
    doc = loaded_nlp(sentence)

    # Vergleiche die vorhergesagten Entitäten mit den tatsächlichen Entitäten
    predicted_entities = [(ent.text, ent.label_) for ent in doc.ents] # wird leer sein (war oben schon leer) , benötigen einen größeren Datensatz
    if predicted_entities == entities: # geht vermutlich bei arrays nicht
        print("Entitäten: Vorhersage stimmt überein")
    else:
        print("Entitäten: Vorhersage stimmt nicht überein")



[{'sentence': 'Queen Sansa Stark is the eldest daughter of Lord Eddard Stark and his wife, Lady Catelyn, sister of Robb, Arya, Bran, and Rickon Stark, and "half-sister" of Jon Snow; though truthfully is his cousin', 'relationships': [{'firstEntity': 'Sansa Stark', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntity': 'Sansa Stark', 'relationshipType': 'Siblings', 'secondEntity': 'Jon Snow'}, {'firstEntity': 'Sansa Stark', 'relationshipType': 'Siblings', 'secondEntity': 'Rickon Stark'}, {'firstEntity': 'Jon Snow', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Jon Snow'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Sansa Stark'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Rickon Stark'}, {'firstEntity': 'Rickon Stark', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntit